In [1]:
import pandas as pd
import itertools
import numpy as np
from statistics import mean
import cvxopt
from cvxopt.solvers import qp, options
from cvxopt import matrix, solvers
from cvxopt import blas
import math

In [32]:
#Initializing variables 
homogenous = pd.read_excel('REC_Profiles.xlsx', sheet_name='Homogenous') 
heterogenous = pd.read_excel('REC_Profiles.xlsx', sheet_name='Heterogenous') 

#price of electricity consumption and injection in 2021 in Wallonia
elec_price = {'cons' : 63,
              'inj' : 44} #€/MWh

#distribution and transport fees for households in low voltage 
distribution_fees_lv = 0.08277 #€/kWh
transport_fees_lv = 0.0255354 #€/kWh
injection_fees_lv = 13.66 # €/year

#prosumer tariff - 1kWe +-= 910kWh 
prosumer_tariff = 63.79 #€/kWe


#price of a pv installation in €/MWh
pv_price_kwh = 1.04 # 1040€ pour 1000kWh

#how much  of the installation should be covered by solar panels
cover_cons = 0.8
    
#management cost for the CER (low cost and high cost options) in €/MWh of produced electricity
management_cost_megawh_low = 5
management_cost_megawh_medium = 8.5
management_cost_megawh_high = 12

#DSO = distribution service operator cost, in €/year
dso_cost_low = 23
dso_cost_medium = 231.5
dso_cost_high = 440

#green certificates 
cv_megawh = 0.57 #0,57 green certificates per MWh produced
cv_price = 65 #65€ per green certificate

permutations = list(itertools.permutations(list('ABCDEF')))

#creates a list containing the households 
households = ['A','B','C','D','E','F']

#generates a list with all the sub-coallitions possible
sub_coallitions = ['ABCDEF','ABCDE','ABCDF','ABCEF','ABDEF','ACDEF','BCDEF','ABCD','ABCE','ABCF','ABDE','ABDF','ABEF','ACDE','ACDF','ACEF','ADEF','BCDE','BCDF','BCEF','BDEF','CDEF','ABC','ABD','ABE','ABF','ACD','ACE','ACF','ADE','ADF','AEF','BCD','BCE','BCF','BDE','BDF','BEF','CDE','CDF','CEF','DEF','AB','AC','AD','AE','AF','BC','BD','BE','BF','CD','CE','CF','DE','DF','EF','A','B','C','D','E','F']

In [3]:
#generates columns filled with zero in the length of the homogenous table
def get_cons(string):
    lst_strings = list(string)
    sum_columns = np.zeros(len(heterogenous))
    for char in lst_strings:
        sum_columns += heterogenous[char]
    
    return sum_columns 
## replace homogenous with heterogenous for heterogenous households 

In [4]:
#creates a table 'cons' with the electricity consumption for each sub-coallition of households
cons = pd.DataFrame()

for sub_coal in sub_coallitions:
    cons[sub_coal] = get_cons(sub_coal) 

In [5]:
#creates a dictionary with the total electricity consumption for each sub-coallition
total_cons = {}

for sub_coal in sub_coallitions: 
    total_cons[sub_coal] = cons[sub_coal].sum()

In [6]:
#creates a dictionary with how much photovoltaic panels capacity to install for each sub-coallition

pv_capacity = {}

for sub_coal in sub_coallitions: 
    pv_capacity[sub_coal] = total_cons[sub_coal]*cover_cons

In [7]:
#the pv capacity for individual households cannot exceed 1/6th of the pv capacity for the community as a whole. 
#if households aggregate into a communtiy, they jointly invest in pv capacity and there is therefore no individual limit. 

for x in households: 
    
    if pv_capacity[x] > 1/6 * pv_capacity['ABCDEF']:
        pv_capacity[x] = 1/6 * pv_capacity['ABCDEF']
    else:
        continue

In [8]:
#creates a table with the electricity production for each sub-coallition of households
prod = pd.DataFrame()

for sub_coal in sub_coallitions:
    prod[sub_coal] = pv_capacity[sub_coal]*heterogenous['PV_prod_to_capacity']
    ## replace homogenous with heterogenous for heterogenous households

In [9]:
#creates a table with the net consumption of electricity per 15 min for each sub-coallition
net_cons = pd.DataFrame()

for sub_coal in sub_coallitions:
    net_cons[sub_coal] = cons[sub_coal] - prod[sub_coal]


In [10]:
#creates two dictionnaries, one containing the total electricity demand for each sub-coallition 
#and the other one the total electricity surplus for each sub-coallition
#the demand is the sum of all positive elements in the table "net_cons"
#the surplus is the sum of all negative elements in the table "net_cons"

total_demand = {}
total_surplus = {}

for sub_coal in sub_coallitions: 
    total_demand[sub_coal] = net_cons[sub_coal][net_cons[sub_coal] > 0].sum()
    total_surplus[sub_coal] = net_cons[sub_coal][net_cons[sub_coal] <= 0].sum()

In [11]:
#generates a dictionary with the number of green certificates for each sub-coallition

green_certificates = {}

for sub_coal in sub_coallitions: 
    if pv_capacity[sub_coal] >= 10000:
        green_certificates[sub_coal] = int(pv_capacity[sub_coal]*cv_megawh/1000)
    else: 
        green_certificates[sub_coal] = 0

In [12]:
#cost for individuals households who are not prosumers 

cost_HH = {}

for x in households:
    cost_HH[x] = total_cons[x]*(distribution_fees_lv + transport_fees_lv) + total_cons[x]*elec_price['cons']/1000

In [13]:
#cost for each sub-coallition forming an energy community 

cost = {}

for sub_coal in sub_coallitions:
    cost[sub_coal] = total_demand[sub_coal]*(distribution_fees_lv + transport_fees_lv + elec_price['cons']/1000) + total_surplus[sub_coal] * elec_price['inj']/1000 + management_cost_megawh_low * pv_capacity[sub_coal]/1000 + dso_cost_low + injection_fees_lv + pv_price_kwh/30 * pv_capacity[sub_coal] - green_certificates[sub_coal] * cv_price
    
    
    

In [14]:
#replace the value for households with cost_homogenous_HH in cost_homogenous because they do not have the same cost and benefits as the community
for x in households: 
    cost[x] = cost_HH[x]

In [15]:
#defines the total cost for all households when they are not prosumers and pay their bill individually

total_cost_HH = sum(cost_HH.values())

In [16]:
#the value generated by the community is te difference between the sum of the cost of individual households when they are not part of a community 
#and the cost of the community when all households join.

value_REC = total_cost_HH - cost['ABCDEF']

In [36]:
#individual value - prosumer - capacity based: 

cost_prosumer_capacity = {}

for x in households: 
    cost_prosumer_capacity[x] = prosumer_tariff*pv_capacity[x]/910 +  net_cons[x].sum()*elec_price['cons']/1000 + pv_capacity[x]*pv_price_kwh/30 + injection_fees_lv
    
value_prosumer_capacity = {}

for x in households:
    value_prosumer_capacity[x] = cost[x] - cost_prosumer_capacity[x]

In [34]:
#cost for prosumer with proportional tariffs

cost_prosumer_proportional = {}

for x in households:
    cost_prosumer_proportional[x] = total_demand[x]*(distribution_fees_lv + transport_fees_lv + elec_price['cons']/1000) + total_surplus[x] * elec_price['inj']/1000 + management_cost_megawh_low * pv_capacity[x]/1000 + dso_cost_low + injection_fees_lv + pv_price_kwh/30 * pv_capacity[x] 
    
value_prosumer_proportional = {}

for x in households:
    value_prosumer_proportional[x] = cost[x] - cost_prosumer_proportional[x]
    

In [18]:
#sharing key 1 - Per capita
sharing_per_capita = {}

for x in households: 
    sharing_per_capita[x] = value_REC/6

In [19]:
#sharing key 2 - Per volume
sharing_per_volume = {}

for x in households: 
    sharing_per_volume[x] = value_REC*total_cons[x]/total_cons['ABCDEF'] 

In [20]:
peak_demand_HH = dict(cons[households].iloc[cons['ABCDEF'].idxmax()])

In [21]:
#sharing key 3 - Pro rata of peak demand
sharing_peak_demand = {}

#gets the demand associated with the peak demand in the community 
peak_demand_HH = dict(cons[households].iloc[cons['ABCDEF'].idxmax()])

for x in households: 
    sharing_peak_demand[x] = value_REC*peak_demand_HH[x]/max(cons['ABCDEF'])

In [22]:
#calculates the values for all the sub-coallitions 

values_coal = {}

for key, value in cost.items():
    sum_HH = sum([cost_HH[letter] for letter in key])
    values_coal[key] = sum_HH - value

In [23]:
#calculates the values for all the permutations of all the sub-coallitions 

values_coal_permuts = {}

for key, value in values_coal.items():
    combination = key
    permut = list(itertools.permutations(list(combination)))
    for perm in permut:
        new_key = ''.join(perm)
        values_coal_permuts[new_key] = value

In [24]:
#for all elements of 'permutations', joins them with no space in between. 
# [('A','B','C','D','E','F'),...] becomes [('ABCDEF'),...]

permutations_stringed = [''.join(perm) for perm in permutations]

In [25]:
#calculate the marginal contribution of each household to the community, given all possible permutations.
shapley = {i:{} for i in 'ABCDEF'}

for permut in permutations_stringed:
    for i, hh in enumerate(permut):  
        if i == 0:
            value = values_coal_permuts[hh]
        else:
            value = values_coal_permuts[permut[:i+1]] - values_coal_permuts[permut[:i]]
        
        shapley[hh][permut] = value

In [26]:
#table containing all the marginal contributions of each individuals to each permutations of the overall community.

shapley_df = pd.DataFrame(shapley)

In [27]:
# sharing key 4 - Shapley value
# getting the average of all marginal contributions for each household

sharing_shapley = {column: shapley_df[column].mean() for column in shapley_df}

In [28]:
#help(qp): qp is part of the cvxopt package wich allows to run a quadratic optimazation program.
#The program looks for a vector of variables x that minimizes the following objective function:
# (1/2)x'Px + q'x
#s.t.
# Gx <= h 
# Ax = b 



M = np.array([[1/math.sqrt(6),1/math.sqrt(6),1/math.sqrt(6),1/math.sqrt(6),1/math.sqrt(6),1/math.sqrt(6)],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0]])
P = matrix(np.dot(M.T,M))
q = matrix(np.array([0,0,0,0,0,0]).T.astype(float))

A = matrix(np.ones((1, 6)))
b = matrix(values_coal['ABCDEF'])

#create a matrix G which has value '-1' attributed to a household when they are part of a (sub-)coallition and '0' when they are not
dicto = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5}
G = []
for sub_coallition in sub_coallitions:
    row = []
    pos_value = []
    for letter in sub_coallition:
        pos_value.append(dicto[letter])
    for key, value in dicto.items():
        if value in pos_value:
            row.append(-1.0)
        else:
            row.append(0.0)
    G.append(row)
G = matrix(np.array(G))
G = matrix(G)

#create a matrix h with the values of all sub-coallitions, multiplied by (-1) to respect the structure for the conditions of the program
h = matrix(np.array(list(values_coal.values())) *(-1))

#Runs the program

sol = qp(P,q,G,h,A,b)['x']

     pcost       dcost       gap    pres   dres
 0:  2.6952e+05 -5.0107e+06  7e+06  4e-02  6e+04
 1:  2.6952e+05 -2.9980e+06  4e+06  2e-02  3e+04
 2:  2.6952e+05 -1.4822e+06  2e+06  1e-02  2e+04
 3:  2.6952e+05 -5.7857e+05  2e+06  8e-03  1e+04
 4:  2.6952e+05  4.3179e+06  2e+06  8e-03  1e+04
 5:  2.6952e+05  5.8511e+07  4e+06  8e-03  1e+04
 6:  2.6952e+05  1.6751e+09  2e+07  8e-03  1e+04
 7:  2.6952e+05  1.6341e+11  3e+08  8e-03  1e+04
 8:  2.6952e+05  1.2164e+13  2e+10  8e-03  1e+04
 9:  2.6952e+05  5.8254e+15  2e+12  8e-03  1e+04
10:  2.6952e+05  9.3210e+18  1e+15  8e-03  2e+05
Terminated (singular KKT matrix).


In [29]:
#sharing key 5 - MinVar 

list(sol)
sharing_minvar = {k: v for (k, v) in zip(dicto.keys(), list(sol))}

In [37]:
#Table with all the allocation keys: 

pd.DataFrame({'individual value 1' : value_prosumer_capacity,
              'individual value 2' : value_prosumer_proportional,
              'per capita': sharing_per_capita,
              'per volume': sharing_per_volume,
              'pro rata of peak demand': sharing_peak_demand,
              'Shapley': sharing_shapley,
              'MinVar': sharing_minvar}).T

,A,B,C,D,E,F
individual value 1,351.929194,263.806013,244.394477,812.620186,103.785770,103.838059
individual value 2,118.750660,97.900508,83.178728,244.312515,11.470016,11.600316
per capita,299.735415,299.735415,299.735415,299.735415,299.735415,299.735415
per volume,348.576741,282.090332,262.355279,666.530676,119.403151,119.456311
pro rata of peak demand,327.098339,110.548949,241.552787,534.324733,580.341931,4.545751
Shapley,342.087663,268.727224,242.531381,723.284031,110.120796,111.661395
MinVar,381.745229,246.683895,234.303616,779.499306,77.084552,79.095893


In [31]:
values_coal['ABCDEF']

1798.4124900868978

For homogenous consumers